## Integrate the particles where they feel all NBody forces

In [1]:
import h5py
import tstrippy
from gcs import path_handler as ph
import gcs
import os 
import numpy as np 
import astropy.units as u
import vanilla
import vanilla_loop_over_monte_carlo as vlm
import datetime 
import GCNBody

In [2]:
author = "Salvatore Ferrone"
author_affiliation = "Sapienza University of Rome"
author_email = "salvatore.ferrone@uniroma1.it"
description = "Integrating star-particles with in a globular cluster"

In [3]:
GCname              =   "NGC104"
montecarlokey       =   "monte-carlo-000"
internal_dynamics   =   "isotropic-plummer"
GCorbits_potential  =   "pouliasis2017pii"
MWpotential         =   "pouliasis2017pii"
NP                  =   int(1e2)
T                   =   5e9*u.yr
dt                  =   1e5*u.yr
NSKIP               =   int(100)

In [4]:
attrs = {"GCname":GCname,"montecarlokey":montecarlokey,"internal_dynamics":internal_dynamics,"GCorbits_potential":GCorbits_potential,"MWpotential":MWpotential,"NP":NP,"T":str(T),"dt":str(dt),"NSKIP":NSKIP}

In [5]:
GCnames=list(tstrippy.Parsers.baumgardtMWGCs().data['Cluster'][:])
GCnames.remove(GCname)

In [7]:
GCnames=GCNBody.load_GCnames_except_for_the_target(GCname)

In [9]:
GCNBodyargs=GCNBody.load_arguments(GCnames=GCnames,GCorbits_potential=GCorbits_potential,montecarlokey=montecarlokey)